1. 문서의 내용을 읽는다.

2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림

3. 임베딩 -> 벡터 데이터베이스에 저장

4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색

5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달.

### 1. 문서의 내용을 읽는다.

In [ ]:
#%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {dir(document)}')

full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

document == ['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']
paragraph == 소득세법
paragraph == [시행 2025. 1. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]
paragraph == 기획재정부(재산세제과(양도소득세)) 044-215-4312
paragraph == 기획재정부(소득세제과(근로소득)) 044-215-4216
paragraph == 기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233
paragraph == 기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
paragraph == 
paragraph == 제1장 총칙 <개정 2009. 12. 31.>
paragraph == 
paragraph == 제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
paragraph == [본조신설 2009. 12. 31.]
paragraph == [종전 제1조는 제2조로 이동 <2009. 12. 31.>] ...

In [ ]:
full_text

'소득세법\n[시행 2025. 1. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., ...

### 2. 문서를 쪼갠다.

In [ ]:
# full_text 쪼개기
#%pip install tiktoken

In [35]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text) # 문자를 숫자로 바꾸는 것
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i:i + chunk_size] # 숫자 리스트를 쪼개는 것
        decoded = encoder.decode(chunk) # 숫자 리스트를 다시 텍스트로 바꾸는 것
        text_list.append(decoded)
    return text_list

In [36]:
chunk_list = split_text(full_text, 1500)

In [ ]:
chunk_list # 우리가 원하는 텍스트가 하나씩 들어있는 것.

['소득세법\n[시행 2025. 1. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무) ...

In [ ]:
# 아래 출력된 것 하나하나가 토큰이다.
encoding

[11226,
 64328,
 11734,
 22070,
 198,
 58,
 5637,
 15719, ... ]

In [38]:
# 문서의 토큰 수 : 1
len(encoding)

155455

In [ ]:
decoded

'소득세법\n[시행 2025. 1. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.

### 3. 문서 임베딩

In [43]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
# collection은 rdb에서 table과 같은 개념

collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [45]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [46]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [47]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [48]:
len(id_list)

104

In [49]:
len(chunk_list)

104

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

### 4. 유사도 검색

In [55]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_doc = tax_collection.query(query_texts=query)

In [ ]:
retrieved_doc['documents'][0] # default가 10이어서 10개를 가져오는 것.

['지일시금\n19. 다음 각 목의 어느 하나에 해당하는 인적용역(제15호부터 제17호까지의 규정을 적용받는 용역은 제외한다)을 일시적으로 제공하고 받는 대가\n가. 고용관계 없이 다수인에게 강연을 하고 강연료 등 대가를 받는 용역\n나. 라디오ㆍ텔레비전방송 등을 통하여 해설ㆍ계몽 또는 연기의 심사 등을 하고 보수 또는 이와 유사한 성질의 대가를 받는 용역\n다. 변호사, 공인회계사, 세무사, 건축사, 측량사, 변리사, 그 밖에 전문적 지식 또는 특별한 기능을 가진 자가 그 지식 또는 기능을 활용하여 보수 또는 그 밖의 대가를 받고 제공하는 용역\n라. 그 밖에 고용관계 없이 수당 또는 이와 유사한 성질의 대가를 받고 제공하는 용역\n20. 「법인세법」 제67조에 따라 기타소득으로 처분된 소득\n21. 제20조의3제1항제2호나목 및 다목의 금액을 그 소득의 성격에도 불구하고 연금외수령한 소득\n22. 퇴직 전에 부여받은 주식매수선택권을 퇴직 후에 행사하거나 고용관계 없이 주식매수선택권을 부여받아 이를 행사함으로써 얻는 이익\n22의2. 종업원등 또는 대학의 교직원이 퇴직한 후에 지급받는 직무발명보상금\n23. 뇌물\n24. 알선수재 및 배임수재에 의하여 받는 금품\n25. 삭제<2020. 12. 29.>\n26. 종교관련종사자가 종교의식을 집행하는 등 종교관련종사자로서의 활동과 관련하여 대통령령으로 정하는 종교단체로부터 받은 소득(이하 “종교인소득”이라 한다)\n27. 「가상자산 이용자 보호 등에 관한 법률」 제2조제1호에 따른 가상자산(이하 “가상자산”이라 한다)을 양도하거나 대여함으로써 발생하는 소득(이하 “가상자산소득”이라 한다)\n② 제1항 및 제19조제1항제21호에도 불구하고 대통령령으로 정하는 서화(書畵)ㆍ골동품의 양도로 발생하는 소득(사업장을 갖추는 등 대통령령으로 정하는 경우에 발생하는 소득은 제외한다)은 기타소득으로 한다.<신설 2020. 12. 29.>\n③ 기타소득금액은 해당 과세기간의 총수입금액에서 이에 사용된 필요경비를 공제한 금액으로 한다.<개정 2020. 12. 29.>\n④ 제1항제26호에 따른 종교인소득에 대하여 제20조제1항에 따른 근로소득으로 원천징수하거나 과세표준확정신고를 한 경우에는 해당 소득을 근로소득으로 본다.<신설 2015. 12. 15., 2020. 12. 29.>\n⑤ 기타소득의 구체적 범위 및 계산방법과 그 밖에 필요한 사항은 대통령령으로 정한다.<개정 2015. 12. 15., 2020. 12. 29.>\n[전문개정 2009. 12. 31.]\n[시행일: 2027. 1. 1.] 제21조제1항제27호\n\n제22조(퇴직소득) ① 퇴직소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2013. 1. 1.>\n1. 공적연금 관련법에 따라 받는 일시금\n2. 사용자 부담금을 기초로 하여 현실적인 퇴직을 원인으로 지급받는 소득\n3. 그 밖에 제1호 및 제2호와 유사한 소득으로서 대통령령으로 정하는 소득\n② 제1항제1호에 따른 퇴직소득은 2002년 1월 1일 이후에 납입된 연금 기여금 및 사용자 부담금을 기초로 하거나 2002년 1월 1일 이후 근로의 제공을 기초로 하여 받은 일시금으로 한다.<개정 2013. 1. 1.>\n③퇴직소득금액은 제1항 각 호에 따른 소득의 금액의 합계액(비과세소득의 금액은 제외한다)으로 한다. 다만, 대통령령으로 정하는 임원의 퇴직소득금액(제1항제1호의 금액은 제외하며, 2011년 12월 31일에 퇴직하였다고 가정할 때 지급받을 대통령령으로 정하는 퇴직소득금액이 있는 경우에는 그 금액을 뺀 금액을 말한다)이 다음 계산식에 따라 계산한 금액을 초과하는 경우에는 제1항에도 불구하고 그 초과하는 금액은 근로소득으로 본다.<개정 2012. 1. 1., 2013. 1. 1., 2014. 12. 23., 2019. 12. 31.>\n\n④ 제3항 단서와 그 계산식을 적용할 때 근무기간과 총급여는 다음 각 호의 방법으로 산정한다.<개정 2014. 12. 23.>\n1. 근무기간: 개월 수로 계산한다. 이 경우 1개월 미만의 기간이 있는 경우에는 이를 1개월로 본다.\n2. 총급여: 봉급ㆍ상여 등 제20조제1항제1호 및 제2호에 따른 근로소득(제12조에 따른 비과세소득은 제외한다)을 합산한다.\n⑤ 삭제<2013. 1. 1.>\n⑥ 퇴직소득의 범위 및 계산방법과 그 밖에 필요한 사항은 대통령령으로 정한다.\n[전문개정 2009. 12. 31.]\n\n제23조 삭제 <2006. 12. 30.>\n\n제3절 소득금액의 계산 <개정 2009. 12. 31.>\n\n제1관 총수입금액 <개정 2009. 12. 31.>\n\n제24조(총수입금액의 계산) ① 거주자의 각 소득에 대한 총수입금액',
' ... ']

### 5. LLM 질의

In [59]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [ ]:
response.choices[0].message.content # '연봉 5천만원인 직장인의 소득세는 얼마인가요?'에 대한 답변

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 과세표준을 계산하고, 그에 따른 기본세율을 적용하여 소득세를 계산해야 합니다. 과세표준은 총급여에서 각종 공제를 적용한 금액입니다.\n\n1. **총급여에서 공제되는 항목들:**\n   - 근로소득공제 \n   - 기본공제 (본인, 배우자, 부양가족 등)\n   - 기타 인적공제 및 특별공제 (예: 의료비, 교육비, 기부금 등)\n\n2. **근로소득공제:**\n   - 연봉 2,000만원 초과 ~ 4,500만원 이하: 100만원 + (총급여액 - 2,000만원) × 35%\n   - 연봉 4,500만원 초과 ~: 975만원 + (총급여액 - 4,500만원) × 5%\n\n따라서, 연봉 5,000만원에 대한 근로소득공제는:\n```\n975만원 + (5,000만원 - 4,500만원) × 5%\n= 975만원 + 25만원\n= 1,000만원\n```\n\n3. **과세표준 계산:**\n   총급여액 5,000만원에서 근로소득공제 1,000만원을 공제:\n   ```\n   5,000만원 - 1,000만원 = 4,000만원\n   ```\n\n4. **기본세율:**\n   과세표준에 따라 기본세율을 적용합니다. 4,000만원에 대한 기본세율은 다음과 같이 적용됩니다:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15% \n   - 따라서, (4,000만원 - 1,200만원) × 15% + 72만원 (이전 구간 세율) + 누진공제액\n\n5. **세액 계산:**\n   ```\n   (4,000만원 - 1,200만원) × 15% + 72만원 \n   = 2,800만원 × 15% + 72만원 \n   = 420만원 + 72만원\n   = 492만원\n   ```\n\n위의 계산은 기본적인 가정 하에 이루어진 것이며, 개인의 상황에 따라 인적공제나 특별공제 등이 적용될 수 있으므로 실제 세액은 다를 수 있습니다. 정확한 계산을 위해서는 소득세 신고 시 상세한 공제 항목을 반영해야 합니다.'